# TIA Comments Ingestor

In [48]:
from pymongo import MongoClient, ASCENDING as asc
import requests
import json
import os
import time

## Setup Clients

In [52]:
# create client for mongo
client = MongoClient('localhost',
                     username='root',
                     password='example')

# get db from client
db = client.tia

# get collection from db
posts_collection = db.posts
comments_collection = db.comments

## Ingestion

In [74]:
# get post ids from mongodb
posts = posts_collection.find({'comments_count': {"$gte": 1}}, {'_id': 1, 'comments_count': 1}, limit=25).sort([('date_gmt', asc)])

for post in posts:
    # parameter for the request
    url = f"https://www.techinasia.com/wp-json/techinasia/2.0/posts/{post['_id']}/comments"
    start_page = 1 # NOTE: maximum page only accept 30
    end_page = 3 # NOTE: maximum page only accept 30
    per_page = 30 # NOTE: maximum per_page only accept 30

    comments_count = 0
    next_page = True

    for page in range(start_page, end_page+1):
        jsonr = requests.get(url, params={'page': start_page, 'per_page': per_page}, headers={'User-Agent': 'Tia-ETL'}).json()

        if len(jsonr['comments']) > 1:
            comments = [{'_id': comment['id']} | comment for comment in jsonr['comments']]
            # print(len(comments))
            for comment in comments:
                # print('inserting to comments collection')
                comments_collection.replace_one({'_id': comment['_id']}, comment, upsert=True)

            comments_count = comments_count + len(comments)

        if (jsonr['total_pages'] <= 1):
            break

    # update the post to add comments_count
    posts_collection.update_one({'_id': post['_id']}, { "$set": {'comments_count': comments_count} })

    time.sleep(1)


## Monitoring Queries

### check and get posts `comments_count` greater or equal than `1`, ordered by `date_gmt`

In [65]:
posts = posts_collection.find({'comments_count': {"$gte": 1}}, {'_id': 1, 'comments_count': 1, 'date_gmt': 1}).sort([('date_gmt', asc)]).limit(10)

for post in posts:
    print(post)

{'_id': '751393', 'date_gmt': '2022-06-06T09:30:15', 'comments_count': 1}
{'_id': '750398', 'date_gmt': '2022-06-07T05:00:24', 'comments_count': 2}
{'_id': '752167', 'date_gmt': '2022-06-08T10:41:24', 'comments_count': 1}
{'_id': '301037', 'date_gmt': '2022-06-08T10:45:20', 'comments_count': 20}
{'_id': '752636', 'date_gmt': '2022-06-10T10:36:11', 'comments_count': 1}
{'_id': '752550', 'date_gmt': '2022-06-13T02:00:46', 'comments_count': 1}
{'_id': '753293', 'date_gmt': '2022-06-15T09:30:45', 'comments_count': 1}
{'_id': '753886', 'date_gmt': '2022-06-16T11:08:48', 'comments_count': 1}
{'_id': '607177', 'date_gmt': '2022-06-17T02:00:00', 'comments_count': 7}
{'_id': '754258', 'date_gmt': '2022-06-19T13:44:30', 'comments_count': 1}


### check and get posts that has no comments

In [67]:
posts = posts_collection.find({'comments_count': {"$lt": 1}}, {'_id': 1, 'comments_count': 1}, limit=10)

for post in posts:
    print(post)

{'_id': '759649', 'comments_count': 0}
{'_id': '759637', 'comments_count': 0}
{'_id': '759348', 'comments_count': 0}
{'_id': '759608', 'comments_count': 0}
{'_id': '759595', 'comments_count': 0}
{'_id': '759579', 'comments_count': 0}
{'_id': '759572', 'comments_count': 0}
{'_id': '759423', 'comments_count': 0}
{'_id': '759552', 'comments_count': 0}
{'_id': '759309', 'comments_count': 0}


### aggregating comments collection by grouping `post` and count each comments

In [86]:
from bson.son import SON

pipeline = [
    {
        "$group": {"_id": "$post", "comments_count": {"$sum": 1}}
    },
    {
        "$sort": SON([("comments_count", -1), ("_id", 1)])
    }
]

comments = comments_collection.aggregate(pipeline)

for comment in comments:
    print(comment)

{'_id': '301037', 'comments_count': 14}
{'_id': '504440', 'comments_count': 9}
{'_id': '607177', 'comments_count': 6}
{'_id': '681468', 'comments_count': 4}
{'_id': '487420', 'comments_count': 2}
{'_id': '612934', 'comments_count': 2}
{'_id': '638604', 'comments_count': 2}


### joins

In [99]:
results = posts_collection.aggregate([
    {
        '$lookup': {
            'from': 'comments', 
            'localField': 'id', 
            'foreignField': 'post', 
            'as': 'joinedResult'
        },
        "$group": {"_id": "$post", "comments_count": {"$sum": 1}}
    }
])

for r in results:
    print(r)

OperationFailure: A pipeline stage specification object must contain exactly one field., full error: {'ok': 0.0, 'errmsg': 'A pipeline stage specification object must contain exactly one field.', 'code': 40323, 'codeName': 'Location40323'}